# Face Recognition Model Training## Using UWA HSFD Hyperspectral DatabaseThis notebook demonstrates building a face recognition model using the UWA HSFD hyperspectral database.Note: Update the data_path below to your local path.

In [ ]:
# Set the data path to your UWA HSFD datasetdata_path = 'path/to/your/UWA_HSFD'  # Replace with your pathprint('Data path set to:', data_path)